блокнот запускался на четырех L4

In [ ]:
import os
import gc
import ctypes
import warnings

import torch
from vllm import LLM, SamplingParams

Настраиваем окружение

In [2]:
warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"]   = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['CMAKE_ARGS'] = "-DGGML_CUDA=on"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [ ]:
pip install einops decord

Создаем device_map, разделяем слои по гпу, что бы избежать проблемы с тензорами на разных устройствах первый и последний слой ставим на одно устройство

In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModel

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    num_layers = {
        'InternVL2-1B': 24, 'InternVL2-2B': 24, 'InternVL2-4B': 32, 'InternVL2-8B': 32,
        'InternVL2-26B': 48, 'InternVL2-40B': 60, 'InternVL2-Llama3-76B': 80}[model_name]

    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

загружаем internVL2

In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModel
path = "OpenGVLab/InternVL2-4B"
device_map = split_model('InternVL2-4B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

Приведение изображение к определенному формату, подгон изображения к определенному соотношению сторон

In [7]:


IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


#обработка входного изображения

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = image_file.convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

загрузка квантованой qwen2.5 с помощью vllm, резервируем для нее часть каждой гпу

In [ ]:

model_path = "/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1"
llm = LLM(
    model_path,
    dtype="half",                
    max_num_seqs=8,            
    max_model_len=1024,          
    trust_remote_code=True,      
    tensor_parallel_size=4,      
    gpu_memory_utilization=0.675,
)

найстройка параметров генерации

In [9]:
sampling_params = SamplingParams(
    temperature=0.75,            
    seed=1,                   
    skip_special_tokens=False,     
    max_tokens = 256,
    min_tokens = 32,
)

In [ ]:
pip install pyTelegramBotAPI

Модель, которая определяет начальный тип задачи

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from PIL import Image

tokenizer_bert = AutoTokenizer.from_pretrained("cointegrated/rubert-base-cased-nli-threeway")
text_classifier = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-base-cased-nli-threeway")
image_classifier = pipeline(task="zero-shot-image-classification", model="google/siglip-so400m-patch14-384")

candidate_labels_ru = ["одежда", "продукт", "цветы", "машины", "другое"]
candidate_labels_eng = ["clothes", "product", "flowers", "cars", "another"]

def zero_shot_classification(sequence_to_classify, image_to_classify):
    tokens = tokenizer_bert([sequence_to_classify] * len(candidate_labels_ru), candidate_labels_ru, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(text_classifier(**tokens.to(text_classifier.device)).logits, -1)
    proba = result[:, 0].cpu().numpy()
    proba /= sum(proba)
    label_idx = proba.argmax()
    if label_idx == 4:
        image_output = image_classifier(image_to_classify, candidate_labels=candidate_labels_eng)
        label = image_output[0]['label']
        label_idx = candidate_labels_eng.index(label)
    return label_idx

Логи

In [3]:
def logging(bot, message, text, is_logging=True):
    if(is_logging):
        bot.send_message(message.chat.id, "[Log]"+ text)

обрабатываем изображение с помощью internVL

In [14]:
def request_to_VLM_internvl(img,prompt_type=1):
    pixel_values = load_image(img, max_num=12).to(torch.bfloat16).to(device)
    generation_config = dict(max_new_tokens=128, do_sample=True)
    
    prompt = {}
    prompt[0] = '''<image>\n 
    you are an internationally awarded specialist in clothing recognition
    prompt generation rules
    1) you must carefully analyze the image
    2) you must identify all the elements of clothing
    3) you must describe only the basic properties of each item of clothing
    4) you must return the response in json format, where the key is description,
    and the value is an array of descriptions of clothing items
    5) the answer must contain information about clothes
    6) you must follow the rules'''

    prompt[1] = '''you have to assume what is most likely in the image, then make a description of the main properties of this object'''
    
    response = model.chat(tokenizer, pixel_values, prompt[prompt_type], generation_config)
    return response

запрос к qwen2.5 72B

In [ ]:
def response_to_llm_qwen(prompt):
    msgs = [
    {"role": "user", "content": prompt}
    ]
    
    response = llm.chat(msgs, sampling_params, use_tqdm=False)
    
    return response[0].outputs[0].text

Парсер перекрестка

In [5]:
import requests
from bs4 import BeautifulSoup
import re

def clean_price(price):
    return re.sub(r"[^\d.,]", "", price)

def get_products_text(query, num_of_products=10):
    response = requests.get(f'https://www.perekrestok.ru/cat/search?search={query}')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = ['https://www.perekrestok.ru' + link['href'] for link in soup.find_all('a', class_='product-card__link')]
    prices = [clean_price(price.text) for price in soup.find_all('div', class_='price-new')]
    titles = [title.text for title in soup.find_all('div', class_='product-card__title')]
    
    text = ''
    for link, price, title in list(zip(links, prices, titles))[:num_of_products]:
        text += f'ТОВАР:\nНазвание - {title}\nЦена - {price} руб.\nСсылка - {link}\n\n'    
    return text

Работа с изображением с помощью gemini

In [ ]:
import requests
import mimetypes
import os
import json
def request_to_VLM(img,promt_type):
    prompt = {}
    prompt[0] = '''<image>\n 
    you are an internationally awarded specialist in clothing recognition
    prompt generation rules
    1) you must carefully analyze the image
    2) you must identify all the elements of clothing
    3) you must describe only the basic properties of each item of clothing
    4) you must return the response in json format, where the key is description,
    and the value is an array of descriptions of clothing items
    5) the answer must contain information about clothes
    6) you must follow the rules
    7) ты должен дать ответ на английском'''

    prompt[1] = '''you have to assume what is most likely in the image, then make a description of the main properties of this object, ты должен дать ответ на английском'''
    
    BASE_URL = "https://generativelanguage.googleapis.com/"  
    GOOGLE_API_KEY = "апи ключ"
    DISPLAY_NAME = "Sample Image"
    
    #  Загрузка тестового изображения
    IMG_PATH_2 = "sample_image.jpg"
    img.save(IMG_PATH_2, format="JPEG")
    
    # Определение MIME-типа и размера файла
    mime_type = mimetypes.guess_type(IMG_PATH_2)[0]
    num_bytes = os.path.getsize(IMG_PATH_2)
    
    # инициализация загрузки для получения URL
    
    headers = {
        "X-Goog-Upload-Protocol": "resumable",
        "X-Goog-Upload-Command": "start",
        "X-Goog-Upload-Header-Content-Length": str(num_bytes),
        "X-Goog-Upload-Header-Content-Type": mime_type,
        "Content-Type": "application/json"
    }
    data = {"file": {"display_name": DISPLAY_NAME}}
    
    response = requests.post(
        f"{BASE_URL}/upload/v1beta/files?key={GOOGLE_API_KEY}",
        headers=headers,
        json=data
    )
    
    # Получаем URL для загрузки данных
    upload_url = response.headers.get("X-Goog-Upload-URL")
    if not upload_url:
        raise Exception("Upload URL not found in response headers")
    
    # Шаг 2: Загрузка самого файла
    with open(IMG_PATH_2, "rb") as file_data:
        upload_headers = {
            "Content-Length": str(num_bytes),
            "X-Goog-Upload-Offset": "0",
            "X-Goog-Upload-Command": "upload, finalize"
        }
        upload_response = requests.post(upload_url, headers=upload_headers, data=file_data)
    
    # Получаем URI загруженного файла из ответа
    file_info = upload_response.json()
    file_uri = file_info.get("file", {}).get("uri")
    if not file_uri:
        raise Exception("File URI not found in upload response")
    
    # Генерация ответа с использованием загруженного файла
    content_request = {
        "contents": [{
            "parts": [
                {"text": prompt[promt_type]},
                {"file_data": {"mime_type": mime_type, "file_uri": file_uri}}
            ]
        }],
    'generationConfig': {
        'stopSequences': [
            'Title',
        ],
        'temperature': 0.7,
        'maxOutputTokens': 256,
        'topP': 0.8,
        'topK': 10,
    },
    }
    
    content_headers = {"Content-Type": "application/json"}
    content_response = requests.post(
        f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={GOOGLE_API_KEY}",
        headers=content_headers,
        json=content_request
    )
    
    response_data = content_response.json()
    
    # Извлечение текста из кандидатов
    return response_data["candidates"][0]["content"]["parts"][0]["text"]

работа с генерацией текста через gemini

In [ ]:
def response_to_LLM(text):
    headers = {
        'Content-Type': 'application/json',
    }
    
    params = {
        'key': 'апи ключ',
    }
    
    json_data = {
        'contents': [
            {
                'parts': [
                    {
                        'text': text,
                    },
                ],
            },
        ],
    'generationConfig': {
        'stopSequences': [
            'Title',
        ],
        'temperature': 0.7,
        'maxOutputTokens': 512,
        'topP': 0.8,
        'topK': 10,
    },
    }
    
    response = requests.post(
        'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent',
        params=params,
        headers=headers,
        json=json_data,
    )
    return response.json()["candidates"][0]["content"]["parts"][0]["text"]
prompt = '''ты должен шоппинг ассистент и твоя задача предложить пользователю какие-то товары, запросы пользователя могут координально отличатся, поэтому ты должен определить задачу для llm модели, твой ответ должен состоять из 2 частей: task (с помощью этого запроса модель определит какие товары нужно найти), subtask(нужен для генерации финального ответа после определения списка товаров, с помощью этой задачи модель предлагает пользователю товары и выполяет просьбу в его запросе)
пример: 'что я могу съесть вместе с этим? на изображении находится хлеб' ты должен ответить так  {"task": "ты должен порекомендовать пользователю несколько вариантов еды или напитков, с которыми можно съесть хлеб" ,"subtask": "ты должен кратко рассказать почему пользователю нужны эти товары"}, пример: пользователь 'я хочу приготовить плов', тогда твой ответ это: {"task":"сделай список товаров, из которого состоит плов", "subtask":"ты должен рассказать пользователю рецепт" } вот информацию про изображение и запрос пользователя: 
текст:Что я могу съесть с этим?
тип задачи: продукт
ответ VLM: The image shows a sliced sausage. It is a boiled sausage, also known as a cooked sausage. It is a cylindrical shape, with a pink color and a smooth texture. The sausage is wrapped in a plastic casing. The label on the sausage indicates that it is a "Doctor's" sausage, which is a type of boiled sausage that is typically made with pork and beef. The sausage is also labeled as being "GOST" certified, which means that it meets Russian quality standards.'''
response_to_LLM(prompt)

'```json\n{\n  "task": "Ты должен порекомендовать пользователю несколько вариантов блюд или продуктов, которые можно съесть с вареной колбасой. Учитывай, что колбаса на изображении - докторская.",\n  "subtask": "Ты должен кратко описать, почему эти варианты сочетаются с вареной колбасой. Например, можно упомянуть о вкусовых сочетаниях, традиционных комбинациях или возможности создания легкого перекуса или полноценного обеда."\n}\n``` \n'

Форматирование генерации ллмок в json

In [45]:
import json
def json_format(text):
    if (len(text.split('```'))>1):
        text = text.split('```')[1].replace('json','')
    return json.loads(text)
pr ='''
информация про пользователя: мужчина, возраст 35 лет, холостой, доход выше среднего
текст:Что я могу съесть с этим?
тип задачи: продукт
ответ VLM: The image shows a sliced sausage. It is a boiled sausage, also known as a cooked sausage. It is a cylindrical shape, with a pink color and a smooth texture. The sausage is wrapped in a plastic casing. The label on the sausage indicates that it is a "Doctor's" sausage, which is a type of boiled sausage that is typically made with pork and beef. The sausage is also labeled as being "GOST" certified, which means that it meets Russian quality standards.'''
print(json_format(response_to_LLM(prompt_llm(1) + pr)))


{'task': 'Найди список продуктов, которые можно съесть с варёной колбасой, учитывая, что пользователь - мужчина, холостой, 35 лет и с доходом выше среднего.', 'subtask': "Предложи несколько вариантов блюд, которые можно приготовить с варёной колбасой, учитывая вкусовые предпочтения мужчины.  Опиши каждый вариант кратко, например, 'Это классическое сочетание, которое идеально подходит для быстрого перекуса.  ' или 'Это более изысканный вариант, который идеально подходит для ужина.' "}


промты

In [57]:
def prompt_llm(index):
    prompts = [
'''
правила генерации ответа
        1) ты должен должен подобрать список продуктов, наиболее подходящих под запрос пользователя
        2) ты должен сформировать поисковый запрос для сайта магазина для каждого проодукта на русском, 
        3) поисковый запрос должен быть простым, нельзя использовать сложные формулировки и спец символы, в одном запросе должен быть один продукт
        4) поисковой запрос должен быть на русском языке
        5) ответ должен быть в формате json в виде {"search_offers": [{"item": "запрос"},{"item": "запрос"},{"item": "запрос"}]}
        6) ответ должен состоять только из ответа в формате 
        7) ты должен соблюдать правила генерации ответа
''',
'''ты шоппинг ассистент, твоя задача предложить пользователю какие-то товары, запросы пользователя могут координально отличатся, поэтому ты должен определить задачу для llm модели, твой ответ должен состоять из 2 частей: task (с помощью этого запроса модель определит какие товары нужно найти), subtask(нужен для того, что бы модель опираясь на найденные товары сгенерировала ответ для пользователя)
пример: 'что я могу съесть вместе с этим? на изображении находится хлеб' ты должен ответить так  {"task": "ты должен порекомендовать пользователю несколько вариантов еды или напитков, с которыми можно съесть хлеб" ,"subtask": "ты должен кратко рассказать почему пользователю нужны эти товары"}, пример: пользователь 'я хочу приготовить плов', тогда твой ответ это: {"task":"сделай список товаров, из которого состоит плов", "subtask":"ты должен рассказать пользователю рецепт" } 
ты должен дать ответ в заданном формате, даже если информации мало, задача по умолчанию это рекомендация наиболее подходящих товаров
вот информацию про изображение и запрос пользователя: 
''','''
ты шоппинг ассистент
ты должен сформировать финальный ответ, обрати внимание на запрос пользователя, информацию про него, найденные для его запроса товары и информацию про изображения
твой ответ должен быть емким, делай содержание небольшим, но оно должно сохранить основную суть
при предложении товаров, ты должен прикреплять основную информацию и ссылки, помимо этого можешь укахывать цену
вот информация про пользователя:

''']
    
    return prompts[index]

бот (может работать и с локальными ллмками)

In [ ]:
import telebot
from PIL import Image
import pytesseract
import requests
from io import BytesIO

is_logging = False
user_info = 'мужчина, возраст 35 лет, холостой, доход выше среднего'

bot = telebot.TeleBot('апи ключ')
# Обработчик команды /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Привет! Я шопинг ассистент\nТеперь у меня есть глаза, пока я могу рекомендовать еду или бытовые товары\nОтправь фото и сформулируй вопрос, я найду именно то, что тебе нужно)\n\nВся информация о боте в /info")

@bot.message_handler(commands=['info'])
def info(message):
    bot.send_message(message.chat.id, "Наш бот нужен для того, что бы консультировать пользователей и предлагать им товары\n\nДля того, что бы получить ответ, нужно загрузить фото и сформулировать запрос в описании к этому фото\n\n В данный момент он работает с едой и некоторыми домашними товарами, мы интегрировали его только с перекрестком\n\nОсновное решение использует internvl2-8B и квантованный Qwen2-72B, нам удалось локально захостить обе модели одновременно в блокноте кагл на четырех L4\n\nПомимо этого, мы интегрировали их в телеграмм бота, но для лучшей точности и надеждности, бот все еще работает на api gemini. (реализацию решения мы загрузили в наш гитхаб https://github.com/Qacen/Sirius_t_bank_AI)\n\nДалее в рамках нашей задачи, мы реализовали логику, позволяющую модели в несколько этапов сгенерировать ответ\nДля большей персонализации, мы храним информацию про пользователя в llm, пример такой информации вы можете найти в \n/user_info \nВ перспективе создание памяти(сильная суммаризация запросов пользователя)\nКак работает ассистент: изначально VLM анализирует изображение и создает его описание\n\nЗатем мы на основании всех имеющихся факторов создаем промпт в llm и просим сформулировать что от нас хочет пользователь\nПри большой вариантивности задач это значительно улучшает качество ответа\nСледующим шагом является определение списка товаров в формате json\nИспользуя поиск на сайте перекресток мы парсим нужные нам товары\nНаконец, llm формирует финальный ответ и мы отправляем его пользователю со списком найденных товаров\n\nДля того, что бы изменить информацию о пользователе используйте команду \n/user_info_change\n\n/log_on - включение логов\n/log_off - отключение логов")

@bot.message_handler(commands=['user_info'])
def handle_user_info(message):
    global user_info
    bot.send_message(message.chat.id, "в данный момент я имею такую информацию про пользователя:\n\n"+user_info)
# Обработчик любых других сообщений
@bot.message_handler(commands=['user_info_change'])
def handle_user_info(message):
    bot.send_message(message.chat.id, "Пожалуйста, введите информацию, которую вы хотите сохранить:")
    bot.register_next_step_handler(message, save_user_info)
    
@bot.message_handler(commands=['log_on'])
def handle_lof_on(message):
    global is_logging
    is_logging = True
    print(is_logging)
    bot.send_message(message.chat.id, "логи включены")
    
@bot.message_handler(commands=['log_off'])
def handle_log_off(message):
    global is_logging
    is_logging = False
    bot.send_message(message.chat.id, "логи отключены")


def save_user_info(message):
    # Сохраняем введенную информацию в словарь
    global user_info
    user_info = message.text
    bot.send_message(message.chat.id, "Информация успешно сохранена!")

# Обработчик сообщений с фото
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    global user_info
    global is_logging
    bot.send_message(message.chat.id, "Отлично, подожди пару минут, мне нужно подумать и дать тебе лучший ответ!(перед началом работы ознакомься с ботом /info)")
    
    # получаем фото
    file_info = bot.get_file(message.photo[-1].file_id)
    file = requests.get(f"https://api.telegram.org/file/bot{bot.token}/{file_info.file_path}")

    img = Image.open(BytesIO(file.content))
    caption_text = message.caption if message.caption else ""

    
    prompt_to_LLM = '''Ты должен дать персональную рекомендацию для пользователя на основе этой информации: \n'''
    prompt_to_LLM += user_info

    # получаем описание изображения от vlm
    text_vlm = request_to_VLM(img,1)
    logging(bot,message,"ответ VLM: "+ text_vlm,is_logging)
        
    # генерируем первый промпт к ллм, это нужно для более точного определения запроса пользователя
    user_want_prompt = prompt_llm(1)
    user_want_prompt += 'информация про пользователя: ' + user_info + '\n запрос пользователя: ' + caption_text + 'информация из изображения: '+ text_vlm
    user_want = json_format(response_to_LLM(user_want_prompt))
    logging(bot,message,"запрос пользователя: "+ str(user_want),is_logging)
    
    # генерируем подборку товаров для пользователя
    print(user_want['subtask'])
    prompt_to_LLM += "\n "+user_want['task']
    prompt_to_LLM += prompt_llm(0)
    prompt_to_LLM += f"после анализа изображения мы получили такую информацию: \n{text_vlm}\n"
    text_llm = response_to_LLM(prompt_to_LLM) 
    logging(bot,message,"ответ LLM: "+ text_llm,is_logging)
    form_json = json_format(text_llm)
    logging(bot,message,"json массив: "+ str(form_json), is_logging)
    
    # парсим данные с перекрестка
    parsing_text = ''
    for i in form_json['search_offers']:
       final_text += get_products_text(i['item'],1)
    print(parsing)
    logging(bot,message,"найдены товары: "+ parsing)
    
    # генерируем финальный ответ
    prompt_to_final_asnwer = prompt_llm(2)
    prompt_to_final_asnwer += "\n запрос пользователя"+ user_want['subtask']
    prompt_to_final_asnwer += '\nинформация про пользователя: ' + user_info + '\n запрос пользователя: ' + caption_text + 'информация из изображения: '+ text_vlm
    prompt_to_final_asnwer += 'найденные товары: ' + text_llm
    
    final_answer = response_to_LLM(prompt_to_final_asnwer) 
    bot.send_message(message.chat.id, final_answer)

bot.polling()

Пример генерации ответа qwen2-72B

In [7]:
sampling_params = SamplingParams(
    temperature=0.45,            
    seed=1,                   
    skip_special_tokens=False,     
    max_tokens = 256,
    min_tokens = 32,
)

msgs = [
    {"role": "user", "content": "расскажи про ансамбли в машинном обучении"}
]

response = llm.chat(msgs, sampling_params, use_tqdm=False)

print(response[0].outputs[0].text)

Ансамбли (ensemble) в машинном обучении представляют собой методы, которые позволяют объединять несколько моделей машинного обучения для улучшения точности прогнозирования и снижения вероятности переобучения. Ансамбли могут быть основаны на разных подходах, таких как бэггинг (bagging), бустинг (boosting) и стакинг (stacking). Давайте рассмотрим каждый из них подробнее.

1. **Бэггинг (Bagging)**:
   - **Описание**: Бэггинг (bootstrap aggregating) – это метод, при котором несколько экземпляров модели обучены на различных подвыборках исходного набора данных. Эти подвыборки создаются путем случайного отбора наблюдений из исходного набора данных с возвращением (bootstrap sample). После обучения каждая модель делает прогноз, и окончательное предсказание формируется путем голосования (для задач классификации) или усреднения (для задач регрессии).
   - **Пример**: Один из самых известных алгоритмов бэггинга – случайный лес (Random Forest). В случайном лесу каждое дерево принимает решение, и ок

In [ ]:
del llm
clean_memory(deep=True)